In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# for initialization
from ssm.optimizers import adam_step, rmsprop_step, sgd_step, convex_combination
from autograd.scipy.misc import logsumexp
from autograd.tracer import getval
from autograd.misc import flatten
from autograd import value_and_grad

In [ ]:
from ssm.models import HMM

# Introduction
Drift-diffusion models are common models of choice behavior and neural activity in 2AFC decision tasks (Gold and Shadlen, *Annu. Rev. Neurosci.* 2007; Ratcliff, *Neural Computation* 2008), where they are typically fit to the set of choices and reaction times in an experiment. Recent work has fit neural instantiations of the DDM to spiking responses (Latimer et al., *Science* 2015; Zoltowski et al., *bioRxiv* 2018). While many formulations of DDM-like models exist in the literature, here we aim to describe how such models can be instantiated as state-space models. We can then leverage the flexibility of this framework to generalize the models in interesting ways and fit them to neural activity.

# Make a simple drift-diffusion model (DDM)  

The state is a one dimensional variable $x_t \in \mathbb{R}$. There is a discrete state $z_t \in \{\mathsf{ramp}, \mathsf{left bound}, \mathsf{right bound}\}$ that indicates whether the continuous state has reached its threshold (bound). Let $u_t \in \mathbb{R}$ be the input, which specifiy the amount of evidence to the left or right choice, which governs the dynamics of $x_t$. 

Model:
$$
\begin{align*}
z_t &\sim p(x_{t-1}) \\
x_t &= x_{t-1} + V_{z_t} u_t + \epsilon_t 
\end{align*}
$$
where $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$ and the input weights are constrained such that
$$
\begin{align*}
V_{\mathsf{ramp}} &= \beta, \\
V_{\mathsf{left bound}} = V_{\mathsf{right bound}} &= 0.
\end{align*}
$$
The discrete transition probabilities are,
$$
\begin{align*}
\log p(z_t = \textsf{left bound} \mid x_{t-1}) &\propto x_{t-1} - 1 \\
\log p(z_t = \textsf{ramp} \mid x_{t-1}) &\propto 0 \\
\log p(z_t = \textsf{right bound} \mid x_{t-1}) &\propto -x_{t-1} - 1
\end{align*}
$$

In [ ]:
hmm = HMM(K=3, D=1, M=1, observations="diagonal_ar", transitions="recurrent_only")

In [ ]:
hmm.init_state_distn.log_pi0 = np.log([0.01, 0.98, 0.01])

# All the states have dynamics matrix of 1 (identity) and bias of 0
hmm.observations.As = np.ones((3, 1, 1))
hmm.observations.bs = np.zeros((3, 1))
hmm.observations._log_sigmasq = np.log(np.array([1e-5, 1e-3, 1e-5])).reshape((3, 1))
hmm.observations.mu_init = np.zeros((3, 1))
hmm.observations._log_sigmasq_init = np.log(.01 * np.ones((3, 1)))

# They only differ in their input 
beta = 1
hmm.observations.Vs[0] = 0       # left bound
hmm.observations.Vs[1] = beta    # ramp
hmm.observations.Vs[2] = 0       # right bound

In [ ]:
w = 100
hmm.transitions.Ws = np.zeros((3, 1))
hmm.transitions.Rs = np.array([w, 0, -w]).reshape((3, 1))
hmm.transitions.r = np.array([-w, 0, -w])

In [ ]:
# Sample state trajectories
T = 100
u = .015 * np.ones((T, 1))

samples = [hmm.sample(T, input=u) for _ in range(100)]
zs, xs = zip(*samples)

In [ ]:
i = 4
plt.plot(1 - zs[i])
plt.plot(xs[i][:, 0])
plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')

In [ ]:
x_mean = np.mean(xs, axis=0)[:, 0]
x_std = np.std(xs, axis=0)[:, 0]

plt.fill_between(np.arange(T), x_mean-2*x_std, x_mean+2*x_std, color='k', edgecolor="none", alpha=0.2)
plt.plot(x_mean, '-k', lw=3)
for x in xs:
    plt.plot(x, '-k', lw=.5, alpha=.25)

plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')

plt.xlabel("time")
plt.ylabel("$x(t)$")

# Try to fit the model

In [ ]:
from ssm.core import BaseHMM
from ssm.init_state_distns import InitialStateDistribution
from ssm.transitions import _Transitions, RecurrentOnlyTransitions
from ssm.observations import _Observations, AutoRegressiveDiagonalNoiseObservations

class DDMObservations(AutoRegressiveDiagonalNoiseObservations):
    def __init__(self, K, D, M=1, lags=1, beta=1.0, sigmas=np.array([[1e-5], [1e-3], [1e-5]])):
        assert K == 3
        assert D == 1
        assert M == 1
        super(DDMObservations, self).__init__(K, D, M)
        
        # The only free parameters of the DDM are the ramp rate...
        self.beta = beta
        
        # and the noise variances, which are initialized in the AR constructor
        self._log_sigmasq = np.log(sigmas)
        
        # Set the remaining parameters to fixed values
        self._As = np.ones((3, 1, 1))
        self.bs = np.zeros((3, 1))
        self.mu_init = np.zeros((3, 1))
        self._log_sigmasq_init = np.log(.01 * np.ones((3, 1)))

        # They only differ in their input 
        self.Vs[0] = 0            # left bound
        self.Vs[1] = self.beta    # ramp
        self.Vs[2] = 0            # right bound
        
    @property
    def params(self):
        return self.beta, self._log_sigmasq
        
    @params.setter
    def params(self, value):
        self.beta, self._log_sigmasq = value
        mask = np.reshape(np.array([0, 1, 0]), (3, 1, 1))
        self.Vs = mask * self.beta
        
    def initialize(self, datas, inputs=None, masks=None, tags=None):
        pass

    def m_step(self, expectations, datas, inputs, masks, tags, **kwargs):
        _Observations.m_step(self, expectations, datas, inputs, masks, tags, **kwargs)
        

# Do the same for the transition model
class DDMTransitions(RecurrentOnlyTransitions):
    def __init__(self, K, D, M=0, scale=100):
        assert K == 3
        assert D == 1
        assert M == 1
        super(DDMTransitions, self).__init__(K, D, M)

        # Parameters linking past observations to state distribution
        self.Ws = np.zeros((3, 1))
        self.Rs = np.array([scale, 0, -scale]).reshape((3, 1))
        self.r = np.array([-scale, 0, -scale])

    @property
    def params(self):
        return ()
    
    @params.setter
    def params(self, value):
        pass
    
    def initialize(self, datas, inputs=None, masks=None, tags=None):
        pass
    
    def m_step(self, expectations, datas, inputs, masks, tags, **kwargs):
        pass


def DDM(beta=1.0, sigmas=np.array([[1e-3], [1e-3], [1e-3]])):
    K, D, M = 3, 1, 1
    
    # Build the initial state distribution, the transitions, and the observations
    init_state_distn = InitialStateDistribution(K, D, M)
    init_state_distn.log_pi0 = np.log([0.01, 0.98, 0.01])
    transition_distn = DDMTransitions(K, D, M)
    observation_distn = DDMObservations(K, D, M, beta=beta, sigmas=sigmas)
    
    return BaseHMM(K, D, M, init_state_distn, transition_distn, observation_distn)

In [ ]:
sigmas = np.array([[1e-4],[1e-3],[1e-4]])
ddm = DDM(sigmas=sigmas)

# Sample state trajectories
T = 100
N_samples = 100

us = []
zs = []
xs = []

for smpl in range(N_samples):
    u = -.025 + .05 * np.random.rand() * np.ones((T, 1))
    z, x = ddm.sample(T, input=u)
    
    us.append(u)
    zs.append(z)
    xs.append(x)

In [ ]:
x_mean = np.mean(xs, axis=0)[:, 0]
x_std = np.std(xs, axis=0)[:, 0]

plt.fill_between(np.arange(T), x_mean-2*x_std, x_mean+2*x_std, color='k', edgecolor="none", alpha=0.2)
plt.plot(x_mean, '-k', lw=3)
for x in xs:
    plt.plot(x, '-k', lw=.5, alpha=.25)

plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')

plt.xlabel("time")
plt.ylabel("$x(t)$")

In [ ]:
test_ddm = DDM(beta=0.0, sigmas=np.ones((3, 1)))
# test_ddm = DDM()

In [ ]:
lls = test_ddm.fit(xs, inputs=us, num_em_iters=25)

plt.plot(lls)

In [ ]:
print(ddm.observations.beta)
print(ddm.observations.sigmasq)

In [ ]:
print(test_ddm.observations.beta)
print(test_ddm.observations.sigmasq)

In [ ]:
print("True: ", ddm.log_likelihood(xs, inputs=us))
print("Test: ", test_ddm.log_likelihood(xs, inputs=us))

# Try a latent version of the DDM with linear emissions

In [ ]:
from ssm.preprocessing import interpolate_data, pca_with_imputation
from ssm.preprocessing import factor_analysis_with_imputation
from tqdm.auto import trange

In [ ]:
from ssm.emissions import _LinearEmissions, GaussianEmissions, PoissonEmissions
from ssm.core import BaseSwitchingLDS
from ssm.util import ensure_args_are_lists


class DDMGaussianEmissions(GaussianEmissions):
    def __init__(self, N, K, D, M=0, single_subspace=True):
        super(DDMGaussianEmissions, self).__init__(N, K, D, M=M, single_subspace=single_subspace)
        # Make sure the input matrix Fs is set to zero and never updated
        self.Fs *= 0

    # Construct an emissions model
    @property
    def params(self):
        return self._Cs, self.ds, self.inv_etas

    @params.setter
    def params(self, value):
        self._Cs, self.ds, self.inv_etas = value

    @ensure_args_are_lists
    def initialize(self, datas, inputs=None, masks=None, tags=None, num_em_iters=50, num_tr_iters=50):
        
        print("Initializing...")
        print("First with FA using {} steps of EM.".format(num_em_iters))
        fa, xhats, Cov_xhats, lls = factor_analysis_with_imputation(self.D, datas, masks=masks, num_iters=num_em_iters)

        # define objective
        def _objective(params, itr):
            Td = sum([x.shape[0] for x in xhats])
            new_datas = [np.dot(x,params[0].T)+params[1] for x in xhats]
            obj = DDM().log_likelihood(new_datas,inputs=inputs)
            return -obj / Td
        
        # initializeR and r
        R = 0.1*np.random.randn(self.D,self.D)
        r = 0.1*np.random.randn(self.D)
        params = [R,r]
        Td = sum([x.shape[0] for x in xhats])
        
        print("Next by transforming latents to match DDM prior using {} steps of max log likelihood.".format(num_tr_iters))

        state = None
        lls = [-_objective(params, 0) * Td]
        pbar = trange(num_tr_iters)
        pbar.set_description("Epoch {} Itr {} LP: {:.1f}".format(0, 0, lls[-1]))

        for itr in pbar:
            params, val, g, state = sgd_step(value_and_grad(_objective), params, itr, state)
            lls.append(-val * Td)
            pbar.set_description("LP: {:.1f}".format(lls[-1]))
            pbar.update(1)
    
        R = params[0]
        r = params[1]
        
        self.Cs = (fa.W @ np.linalg.inv(R)).reshape([self.D,self.N,self.D])
        self.ds = fa.mean - fa.W @ np.linalg.inv(R) @ r
        self.inv_etas[:,...] = np.log(fa.sigmasq) 
        
#     def invert(self, data, input=None, mask=None, tag=None):
#         return self._invert(data, input=input, mask=mask, tag=tag)
    
#     def _invert(self, data, input=None, mask=None, tag=None):
        # this should return xhat! e.g. for variational inference. use initialization procedure here... 
#         pass
        
def LatentDDM(N, beta=1.0, sigmas=np.array([[1e-5], [1e-3], [1e-5]])):
    K, D, M = 3, 1, 1
    
    # Build the initial state distribution, the transitions, and the observations
    init_state_distn = InitialStateDistribution(K, D, M)
    init_state_distn.log_pi0 = np.log([0.01, 0.98, 0.01])
    transition_distn = DDMTransitions(K, D, M)
    dynamics_distn = DDMObservations(K, D, M, beta=beta, sigmas=sigmas)

    emission_distn = DDMGaussianEmissions(N, K, D, M=M, single_subspace=True)
    
#     @ensure_args_are_lists
#     def log_probability(self, datas, inputs=None, masks=None, tags=None):
#         return self.log_likelihood(datas, inputs, masks, tags) + self.log_prior()

    # Make the SLDS
    return BaseSwitchingLDS(N, K, D, M, init_state_distn, transition_distn, dynamics_distn, emission_distn)

In [ ]:
# Map the latents to "neural" observations
N = 10
C = np.random.rand(N, 1)
# C, _, _ = np.linalg.svd(C, full_matrices=False)

In [ ]:
ys = [np.dot(x, C.T) +1.0 + .01 * np.random.randn(T, N) for x in xs]

In [ ]:
latent_ddm = LatentDDM(N)
latent_ddm.emissions.initialize(ys, inputs=us)

In [ ]:
# dir(latent_ddm.emissions)
# print(latent_ddm.emissions._Cs)
print(latent_ddm.emissions.Cs)
print(C)

In [ ]:
# Create a variational posterior
from ssm.variational import SLDSMeanFieldVariationalPosterior
q_mf = SLDSMeanFieldVariationalPosterior(latent_ddm, ys, inputs=us, initial_variance=0.01)
# q_mf._params = [(np.random.randn(T, 1), np.zeros((T, 1))) for _ in range(N_samples)]
# q_mf._params = [(xs[i], np.log(.01) * np.ones((T, 1))) for i in range(N_samples)]

In [ ]:
q_mf_elbos = latent_ddm.fit(q_mf, ys, inputs=us, method="svi", num_iters=100, initialize=False, step_size=0.1)

# Get the posterior mean of the continuous states
q_mf_x = q_mf.mean[0]

In [ ]:
plt.plot(q_mf_elbos)

In [ ]:
for xhat in q_mf.mean:
    plt.plot(xhat, '-k', lw=1, alpha=.25)

In [ ]:
tr = 1
tr += 1
yhat = latent_ddm.smooth(q_mf.mean[tr], ys[tr], input=us[tr])
plt.figure(figsize=[12,4])
plt.subplot(1,2,1)
plt.plot(xs[tr],'k')
plt.plot(q_mf.mean[tr],'r--')
plt.subplot(1,2,2)
plt.plot(ys[tr])
plt.plot(yhat, '--')

In [ ]:
# latent_ddm.emissions.Cs = C.reshape([1,10,1])
# latent_ddm.emissions.ds = np.zeros([10])
print(latent_ddm.emissions.Cs)
print(latent_ddm.emissions.ds)
print(latent_ddm.emissions.inv_etas)
# print(latent_ddm.emissions.Fs)

In [ ]:
print(ddm.observations.beta)
print(ddm.observations._log_sigmasq)

In [ ]:
print(latent_ddm.dynamics.beta)
print(latent_ddm.dynamics._log_sigmasq)
latent_ddm.dynamics.beta = ddm.observations.beta
latent_ddm.dynamics._log_sigmasq = ddm.observations._log_sigmasq
# print(latent_d)

# Next Steps

The latent model didn't work out of the box -- the initial state trajectories are centered to be mean zero, which isn't a reasonable assumption for this model.  

Next up, let's try elaborations of the model. 

# Time-varying input

Here, the inputs are sequences of counts including zeros. This is intended to simulate the "Poisson clicks" task.

In [ ]:
def generate_clicks(T=1.0,dt=0.01,rate_r=20,rate_l=20):
    """
    This function generates right and left 'clicks' from two Poisson processes with rates rate_r and rate_l
    over T seconds with bin sizes dt. The outputs are binned clicks into discrete time bins.
    """
    
    # number of clicks
    num_r = np.random.poisson(rate_r*T)
    num_l = np.random.poisson(rate_l*T)
    
    # click times
    click_time_r = np.sort(np.random.uniform(low=0.0,high=T,size=[num_r,1]))
    click_time_l = np.sort(np.random.uniform(low=0.0,high=T,size=[num_l,1]))
    
    # binned outputs are arrays with dimensions Tx1
    binned_r = np.histogram(click_time_r,np.arange(0.0,T+dt,dt))[0]
    binned_l = np.histogram(click_time_l,np.arange(0.0,T+dt,dt))[0]

    return binned_r, binned_l

In [ ]:
ddm = DDM(sigmas=np.array([[1e-4], [5e-4], [1e-4]]))

# Sample state trajectories
T = 100 # number of time bins
trial_time = 1.0 # trial length in seconds
dt = 0.01 # bin size in seconds
N_samples = 100 

# input statistics
total_rate = 40 # the sum of the right and left poisson process rates is 40

us = []
zs = []
xs = []

for smpl in range(N_samples):

    # randomly draw right and left rates
    rate_r = np.random.randint(0,total_rate+1)
    rate_l = total_rate - rate_r

    # generate binned right and left clicks
    u_r, u_l = generate_clicks(T=trial_time,dt=dt,rate_r=rate_r,rate_l=rate_l)
    
    # input is sum of u_r and u_l
    u = 0.1*u_r - 0.1*u_l
    u = u[:,np.newaxis]
    z, x = ddm.sample(T, input=u)
    
    us.append(u)
    zs.append(z)
    xs.append(x)
    

In [ ]:
us_test = []
zs_test = []
xs_test = []

for smpl in range(N_samples):

    # randomly draw right and left rates
    rate_r = np.random.randint(0,total_rate+1)
    rate_l = total_rate - rate_r

    # generate binned right and left clicks
    u_r, u_l = generate_clicks(T=trial_time,dt=dt,rate_r=rate_r,rate_l=rate_l)
    
    # input is sum of u_r and u_l
    u = 0.1*u_r - 0.1*u_l
    u = u[:,np.newaxis]
    z, x = ddm.sample(T, input=u)
    
    us_test.append(u)
    zs_test.append(z)
    xs_test.append(x)

In [ ]:
x_mean = np.mean(xs, axis=0)[:, 0]
x_std = np.std(xs, axis=0)[:, 0]

plt.fill_between(np.arange(T), x_mean-2*x_std, x_mean+2*x_std, color='k', edgecolor="none", alpha=0.2)
plt.plot(x_mean, '-k', lw=3)
for x in xs:
    plt.plot(x, '-k', lw=.5, alpha=.25)

plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')

plt.xlabel("time")
plt.ylabel("$x(t)$")

In [ ]:
xs[i].shape

In [ ]:
C = np.sort(np.random.randn(100))[None, :]

In [ ]:
# Plot the latent state and a bunch of linear-nonlinear projections of it
import seaborn as sns
sns.set_context("talk")

from matplotlib.gridspec import GridSpec
i = 2
gs = GridSpec(3, 1, height_ratios=[1, 5, 5])
plt.figure(figsize=(6, 8))

plt.subplot(gs[0])
plt.plot(us[i])
plt.plot(np.zeros(T), ':k')
plt.ylim(-.5, .5)
plt.yticks([0])
plt.ylabel("input")
plt.xlim(0, T)
plt.xticks([])

plt.subplot(gs[1])
plt.plot(xs[i])
plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')
plt.ylim(-1.25, 1.25)
plt.yticks([-1, 0, 1])
plt.ylabel("latent state")
plt.xlim(0, T)
plt.xticks([])

plt.subplot(gs[2])
# y = np.maximum(0, 2 + xs[i].dot(np.random.randn(1, 100)))
y = np.log(1 + np.exp(2 + xs[i].dot(C))) + .25 * np.random.rand(100, 100)
plt.imshow(np.maximum(y.T, 0), aspect="auto", vmin=0, cmap="Greys")
# plt.plot(y)
# plt.ylim(-1.25, 1.25)
# plt.yticks([])
plt.xlim(0, T)
plt.ylabel("firing rate")
plt.tight_layout()
plt.savefig("accumulator_1d_{}.pdf".format(i))

In [ ]:
xs[i]

In [ ]:
test_ddm = DDM(beta=0.0, sigmas=np.ones((3, 1)))

In [ ]:
lls = test_ddm.fit(xs, inputs=us, num_em_iters=25)

plt.plot(lls)

In [ ]:
test_accum = Accumulator(beta=0.0, sigmas=np.ones((3, 1)), As=np.ones((3,1,1)))
test_accum.fit(xs, inputs=us, num_em_iters=25)

In [ ]:
print(ddm.observations.beta)
print(ddm.observations._log_sigmasq)

In [ ]:
print(test_ddm.observations.beta)
print(test_ddm.observations._log_sigmasq)

In [ ]:
print("True: ", ddm.log_likelihood(xs_test, inputs=us_test))
print("Test: ", test_ddm.log_likelihood(xs_test, inputs=us_test))
print("Test: ", test_accum.log_likelihood(xs_test, inputs=us_test))

**Alternative parameterization of DDM with discrete input levels**
Here we consider the DDM for random dot motion (RDM). In RDM tasks, we typically have discrete input levels, and it is useful to consider learning a parameter $V_{z_t}$ for each input level. Let us say we have $C$ coherence levels. We can then parameterize with a one-hot input encoding the coherence level, such that $u_t \in \mathbb{R}^C$. Then, we can learn a $C$ dimensional vector of parameters $V_{z_t}$. The dynamics model in this case would be
$$
\begin{align*}
z_t &\sim p(x_{t-1}) \\
x_t &= x_{t-1} + V_{z_t}^\top u_t + \epsilon_t 
\end{align*}
$$

# Accumulator models
In the next section we develop accumulator models based on Brunton et al., Science 2013. The 1D accumulator model from Brunton 2013 has the form
$$
\begin{align*}
x_t &= 
\begin{cases}
0 & \text{if } |x_{t-1}|>B \\
(1 + \lambda) \, x_{t-1} + \left( \delta_{t,t_R} \eta_R c_t - \delta_{t,t_L} \eta_L c_t \right) + \epsilon_t & \text{otherwise}
\end{cases}
\end{align*}
$$
where $\epsilon_t \sim \mathcal{N}(0,\sigma_a^2)$, $\eta_R \sim \mathcal{N}(1, \sigma_s^2)$, $\eta_L \sim \mathcal{N}(1, \sigma_s^2)$ and 
$$
\begin{align*}
c_t &= \left[ \frac{\tau_\varphi - 1}{\tau_\varphi} + (\varphi - 1) (\delta_{t,t_R} + \delta_{t,t_L}) \right] c_{t-1} + \frac{1}{\tau_\varphi}
\end{align*}.
$$
The boundary $B$ acts in the same mechanism as the boundary in the DDM. The initial value $x_0$ comes from a zero-mean normal distribution with variance $\sigma_i^2$.

We can write the dynamics in the accumulation state (before boundary) with a coupled LDS
$$
\begin{align*}
x_0 & \sim \mathcal{N}(0, \sigma_i^2) \\
x_t & \sim \mathcal{N} \bigg((1 + \lambda) \, x_{t-1} + ( \delta_{t,t_R} -  \delta_{t,t_L} ) c_t , \, \sigma_a^2 + (\delta_{t,t_R}^2 + \delta_{t,t_L}^2)\,  c_t^2 \, \sigma_s^2 \bigg)\\
c_t & \sim \mathcal{N} \bigg(\left[ \frac{\tau_\varphi - 1}{\tau_\varphi} + (\varphi - 1) (\delta_{t,t_R} + \delta_{t,t_L}) \right] c_{t-1} + \frac{1}{\tau_\varphi}, \, \sigma_c^2 \bigg)
\end{align*}
$$

Brunton, B. W., Botvinick, M. M., & Brody, C. D. (2013). Rats and humans can optimally accumulate evidence for decision-making. Science, 340(6128), 95-98.

DePasquale et al., Cosyne 2019.

## 1D Accumulator with decay

We can include accumulator drift via the modified dynamics
$$
\begin{align*}
x_t &= A_{z_t} x_{t-1} + V_{z_t} u_t + \epsilon_t.
\end{align*}
$$
Here, $A_\textsf{ramp}$ corresponds to $1 + \lambda$ from the (Brunton, 2013) accumulator model. We note that in the current implementation of this model, the $A_{z_t}$ are learned for each state.

In [ ]:
class AccumulatorObservations(DDMObservations):
    def __init__(self, K, D, M=1, lags=1, beta=1.0, sigmas=1e-3 * np.ones((3, 1)), As=np.ones((3,1,1))):
        super(AccumulatorObservations, self).__init__(K, D, M, beta=beta, sigmas=sigmas)

        # learn diagonal autoregressive dynamics 
        self._As = As

    @property
    def params(self):
        return self.beta, self._log_sigmasq, self._As

    @params.setter
    def params(self, value):
        self.beta, self._log_sigmasq, self._As = value
        mask = np.reshape(np.array([0, 1, 0]), (3, 1, 1))
        self.Vs = mask * self.beta

    def initialize(self, datas, inputs=None, masks=None, tags=None, covariances=None):
        pass

    def m_step(self, expectations, datas, inputs, masks, tags, covariances, **kwargs):
        _Observations.m_step(self, expectations, datas, inputs, masks, tags, covariances, **kwargs)

    
def Accumulator(beta=1.0, sigmas=np.array([[1e-5], [1e-3], [1e-5]]), As=np.ones((3, 1, 1))):
    K, D, M = 3, 1, 1
    
    # Build the initial state distribution, the transitions, and the observations
    init_state_distn = InitialStateDistribution(K, D, M)
    init_state_distn.log_pi0 = np.log([0.01, 0.98, 0.01])
    transition_distn = DDMTransitions(K, D, M)
    observation_distn = AccumulatorObservations(K, D, M, beta=beta, sigmas=sigmas, As=As)
    
    return BaseHMM(K, D, M, init_state_distn, transition_distn, observation_distn)

In [ ]:
As = np.array([[1.0,0.95,1.0]]).reshape((3,1,1))
accum = Accumulator(sigmas=[[1e-5], [1e-3], [1e-5]], As=As)

In [ ]:
# Sample state trajectories
T = 100 # number of time bins
trial_time = 1.0 # trial length in seconds
dt = 0.01 # bin size in seconds
N_samples = 100 

# input statistics
total_rate = 40 # the sum of the right and left poisson process rates is 40

us = []
zs = []
xs = []

for smpl in range(N_samples):

    # randomly draw right and left rates
    rate_r = np.random.randint(0,total_rate+1)
    rate_l = total_rate - rate_r

    # generate binned right and left clicks
    u_r, u_l = generate_clicks(T=trial_time,dt=dt,rate_r=rate_r,rate_l=rate_l)
    
    # input is sum of u_r and u_l
    u = 0.1*u_r - 0.1*u_l
    u = u[:,np.newaxis]
    z, x = accum.sample(T, input=u)
    
    us.append(u)
    zs.append(z)
    xs.append(x)

In [ ]:
x_mean = np.mean(xs, axis=0)[:, 0]
x_std = np.std(xs, axis=0)[:, 0]

plt.fill_between(np.arange(T), x_mean-2*x_std, x_mean+2*x_std, color='k', edgecolor="none", alpha=0.2)
plt.plot(x_mean, '-k', lw=3)
for x in xs:
    plt.plot(x, '-k', lw=.5, alpha=.25)

plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')

plt.xlabel("time")
plt.ylabel("$x(t)$")

In [ ]:
test_accum = Accumulator(beta=0.0, sigmas=np.ones((3, 1)), As=np.ones((3,1,1)))

In [ ]:
lls = test_accum.fit(xs, inputs=us, num_em_iters=25)

plt.plot(lls)

In [ ]:
test_ddm = DDM(beta=0.0, sigmas=np.ones((3, 1)))
test_ddm.fit(xs, inputs=us, num_em_iters=25)

In [ ]:
print(accum.observations.beta)
print(accum.observations._log_sigmasq)
print(accum.observations._As)

In [ ]:
print(test_accum.observations.beta)
print(test_accum.observations._log_sigmasq)
print(test_accum.observations._As)

In [ ]:
# Sample test data 

T = 100 # number of time bins
trial_time = 1.0 # trial length in seconds
dt = 0.01 # bin size in seconds
N_samples = 100 

# input statistics
total_rate = 40 # the sum of the right and left poisson process rates is 40

us_test = []
zs_test = []
xs_test = []

for smpl in range(N_samples):

    # randomly draw right and left rates
    rate_r = np.random.randint(0,total_rate+1)
    rate_l = total_rate - rate_r

    # generate binned right and left clicks
    u_r, u_l = generate_clicks(T=trial_time,dt=dt,rate_r=rate_r,rate_l=rate_l)
    
    # input is sum of u_r and u_l
    u = 0.1*u_r - 0.1*u_l
    u = u[:,np.newaxis]
    z, x = accum.sample(T, input=u)
    
    us_test.append(u)
    zs_test.append(z)
    xs_test.append(x)

In [ ]:
print("True: ", accum.log_likelihood(xs_test, inputs=us_test))
print("Test: ", test_accum.log_likelihood(xs_test, inputs=us_test))
print("Test: ", test_ddm.log_likelihood(xs_test, inputs=us_test))

# 2D Accumulator

Accumulation to bound with 2 input dimensions: let $u_t \in \mathbb{R}^2$ and $V_{z_t} \in \mathbb{R}^2$.
$$
\begin{align*}
x_t &= (1 + \lambda) x_{t-1} + V_{z_t}^\top u_t + \epsilon_t 
\end{align*}
$$

In [ ]:
# to do -> the V matrix should be D by M, or you should assert that D = M. 

class Accumulator2DObservations(AutoRegressiveDiagonalNoiseObservations):
    def __init__(self, K, D=2, M=0, lags=1, betas=np.ones(2,), sigmas=1e-3 * np.ones((3, 2)), a_diag=np.ones((3,2,1))):
        assert K == 3
        assert D == 2
        super(Accumulator2DObservations, self).__init__(K, D, M)
        
        # dynamics matrix for accumulation state
        self._a_diag = a_diag
        mask = np.array([np.eye(D), np.eye(D), np.eye(D)])
        self._As = self._a_diag*mask
        
        # old version with a_diag a 2-dimensional vector
#         self._As[0] = np.eye(D)
#         self._As[1] = a_diag*np.eye(D)
#         self._As[2] = np.eye(D)

        # set input accumulator params, one for each dimension
        self._betas = betas
        
        # They only differ in their input 
        self.Vs[0] *= np.zeros((D,D))            # left bound
        self.Vs[1] = self._betas*np.eye(D)       # ramp
        self.Vs[2] *= np.zeros((D,D))            # right bound
        
        # set noise variances, which are initialized in the AR constructor
        self._log_sigmasq = np.log(sigmas)
        
        # Set the remaining parameters to fixed values
        self.bs = np.zeros((3, D))
        self.mu_init = np.zeros((3, D))
        self._log_sigmasq_init = np.log(.01 * np.ones((3, D)))
        
    @property
    def params(self):
        return self._betas, self._log_sigmasq, self._a_diag
        
    @params.setter
    def params(self, value):
        D = self.D
        self._betas, self._log_sigmasq, self._a_diag = value
        mask = np.array([np.zeros((D,D)), np.eye(D), np.zeros((D,D))])
        self.Vs = mask * self._betas
        a_mask = np.array([np.eye(D), np.eye(D), np.eye(D)])
#         self._As = a_mask * self._a_diag
        self._As = self._a_diag*a_mask         # old version with a_diag a 2-dimensional vector
        
    def initialize(self, datas, inputs=None, masks=None, tags=None, covariances=None):
        pass

    def m_step(self, expectations, datas, inputs, masks, tags, covariances, **kwargs):
        _Observations.m_step(self, expectations, datas, inputs, masks, tags, covariances, **kwargs)
    

# Transition model
class Accumulator2DTransitions(RecurrentOnlyTransitions):
    def __init__(self, K, D, M=0, scale=100):
        assert K == 3
        assert D == 2
        assert M == 2
        super(Accumulator2DTransitions, self).__init__(K, D, M)
        
        self.Ws = np.zeros((K,M))
        self.Rs = np.array([[scale, -scale], [0, 0], [-scale, scale]]).reshape((K,D)) # K by D
        self.r = np.array([-scale, 0, -scale])
        
    @property
    def params(self):
        return ()
    
    @params.setter
    def params(self, value):
        pass
    
    def initialize(self, datas, inputs=None, masks=None, tags=None, covariances=None):
        pass
    
    def m_step(self, expectations, datas, inputs, masks, tags, covariances, **kwargs):
        pass
    
def Accumulator2D(D=2, M=2, betas=np.ones(2,), sigmas=np.array([[2e-4,1e-4],[3e-4,5e-4],[1e-4,2e-4]]), a_diag=np.ones((3,2,1))):
    K, D, M = 3, 2, 2
    
    # Build the initial state distribution, the transitions, and the observations
    init_state_distn = InitialStateDistribution(K, D, M)
    init_state_distn.log_pi0 = np.log([0.01, 0.98, 0.01])
    transition_distn = Accumulator2DTransitions(K, D, M)
    observation_distn = Accumulator2DObservations(K, D, M, betas=betas, sigmas=sigmas, a_diag=a_diag)
    
    return BaseHMM(K, D, M, init_state_distn, transition_distn, observation_distn)

In [ ]:
# sigmas = np.array([[1e-4], [1e-3], [1e-5]])*np.ones((3,2))
sigmas = np.array([[2e-4,1e-4],[3e-4,5e-4],[1e-4,2e-4]])
betas = np.array([1.1,0.9])
a_diag = np.ones((3,2,1))
# a_diag = np.array([[1.0,1.0],[0.98,0.95],[1.0,1.0]]).reshape((3,2,1))
acc2 = Accumulator2D(sigmas=sigmas,betas=betas,a_diag=a_diag)

# Sample state trajectories
T = 100 # number of time bins
trial_time = 1.0 # trial length in seconds
dt = 0.01 # bin size in seconds
N_samples = 100 

# input statistics
total_rate = 40 # the sum of the right and left poisson process rates is 40

us = []
zs = []
xs = []

for smpl in range(N_samples):

    # randomly draw right and left rates
    rate_r = np.random.randint(0,total_rate+1)
    rate_l = total_rate - rate_r

    # generate binned right and left clicks
    u_r, u_l = generate_clicks(T=trial_time,dt=dt,rate_r=rate_r,rate_l=rate_l)
    
    # input is sum of u_r and u_l
    u = 0.1*np.array([u_r,u_l]).T
    z, x = acc2.sample(T, input=u)
    
    us.append(u)
    zs.append(z)
    xs.append(x)

In [ ]:
tr = 0
# tr += 1
plt.plot(1 - zs[tr])
plt.plot(xs[tr][:, 0], 'b')
plt.plot(xs[tr][:, 1], 'r')
# plt.plot(np.ones(T), ':k')
# plt.plot(-np.ones(T), ':k')

In [ ]:
test_accum = Accumulator2D(betas=np.zeros(2,), sigmas=np.ones((3, 2)), a_diag=0.9*np.ones((3,2,1)))

In [ ]:
lls = test_accum.fit(xs, inputs=us, num_em_iters=25)

plt.plot(lls)

In [ ]:
print(acc2.observations._betas)
print(acc2.observations._log_sigmasq)
print(acc2.observations._a_diag)

print(test_accum.observations._betas)
print(test_accum.observations._log_sigmasq)
print(test_accum.observations._a_diag)

In [ ]:
print("True: ", acc2.log_likelihood(xs, inputs=us))
print("Test: ", test_accum.log_likelihood(xs, inputs=us))

**Other Models**
1. Include adaptation $c_t$
2. Add input-dependent variance from Bing's model
3. Time-lag in inputs - some mechanism to incorporate time-delays from stimulus presentation until neural response.
4. Gain modulation. Add an initial dimension
$$
g_t \sim \mathcal{N}(g_{t-1}, \sigma_g^2)
$$
that modulates the mapping from latent state to neural responses. This could vary slowly within trials and/or more rapidly across trials. 

In [ ]:
choice = np.random.choice(2, size=100)
choice = np.array(sorted(choice))
C = np.random.rand(100, 2) * (np.arange(2)[None, :] == choice[:, None])

In [ ]:
# Plot the latent state and a bunch of linear-nonlinear projections of it
import seaborn as sns
sns.set_context("talk")

from matplotlib.gridspec import GridSpec
i = 1
gs = GridSpec(3, 1, height_ratios=[1, 5, 5])
plt.figure(figsize=(6, 8))

plt.subplot(gs[0])
plt.plot(us[i])
plt.plot(np.zeros(T), ':k')
plt.ylim(-.5, .5)
plt.yticks([0])
plt.ylabel("input")
plt.xlim(0, T)
plt.xticks([])

plt.subplot(gs[1])
plt.plot(xs[i])
plt.plot(np.ones(T), ':k')
plt.plot(-np.ones(T), ':k')
plt.ylim(-1.25, 1.25)
plt.yticks([-1, 0, 1])
plt.ylabel("latent state")
plt.xlim(0, T)
plt.xticks([])

plt.subplot(gs[2])
# y = np.maximum(0, 2 + xs[i].dot(np.random.randn(1, 100)))
# C = np.random.rand(100, 2) * np.random.dirichlet(.1 * np.ones(2), size=(100))
y = np.log(1 + np.exp(2 + xs[i].dot(C.T)))
plt.imshow(np.maximum(y.T, 0), aspect="auto", vmin=0, cmap="Greys")
# plt.plot(y)
# plt.ylim(-1.25, 1.25)
# plt.yticks([])
plt.xlim(0, T)
plt.ylabel("firing rate")
plt.tight_layout()
plt.savefig("accumulator_2d_{}.pdf".format(i))

In [ ]:
plt.imshow(C, aspect="auto")
plt.colorbar()